<a href="https://colab.research.google.com/github/chillburg/sentiment/blob/master/Copy_of_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from time import time
import pandas as pd
import numpy as np
from numpy import array
import random
import nltk

from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation, Flatten
from keras.optimizers import Adam, RMSprop, Adadelta
from keras.utils import normalize
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras.layers import Dropout, Conv2D, MaxPooling2D, Input, Dense, Flatten, concatenate, GlobalMaxPool2D, Lambda
from keras.layers import GlobalAveragePooling2D, Reshape, add, LSTM, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Lambda
import keras.backend as K


# from google.colab import drive
# drive.mount('/content/drive')

Using TensorFlow backend.


## Preparing our data

In [ ]:
df = pd.read_csv('/content/drive/My Drive/BALTA/Sentiment/NPS.csv', dtype=str, encoding="ISO-8859-13")
df.shape
df.head()

,SEQ_NO,STATUS,STATUS_NAME,EVENT_TYPE,PROCESS_ERROR,SURVEY_EXE_DATE,SURVEY_TYPE,SURVEY_KEY,EMAIL_TO,ID_NO,LANGUAGE,POLICY_NO,POLICY_SEQ_NO,POLICY_YEAR,CLA_CASE_NO,CLA_SUBCASE_NO,CREATED_DATE,TIMESTAMP,res_STATUS,res_STATUS_NAME,res_FEEDBACK_ID,CREATED_AT,COMPLETED_AT,SCORE1,SCORE2,TEXT1,TEXT2,TEXT3,TEXT4
0,296,S,"Processed, request sent",N,NaN,27AUG2019:14:36:17,RETAIL_KAC_CC,jkMKKWUFgysGLFsH,vilis.krumins@gmail.com,11948340,LV,591288369,89489544,4,NaN,NaN,25JUL2019:04:00:15,27AUG2019:14:36:17,NaN,NO RESPONSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,301,S,"Processed, request sent",N,NaN,27AUG2019:14:36:17,RETAIL_KAC_CC,nPSjOJZKIMTwkePm,gin488@inbox.lv,11947344,LV,648881169,89468715,1,NaN,NaN,25JUL2019:04:00:15,27AUG2019:14:36:17,NaN,NO RESPONSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,309,S,"Processed, request sent",N,NaN,27AUG2019:14:36:18,RETAIL_KAC_CC,cCdjYKupwAmbfTiC,lebedevainga@inbox.lv,705241,LV,544963686,89489432,6,NaN,NaN,25JUL2019:04:00:15,27AUG2019:14:36:18,NaN,NO RESPONSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,318,S,"Processed, request sent",N,NaN,27AUG2019:14:36:18,RETAIL_KAC_CC,dKzAODqrxgHuBjsJ,signe.rubene@gmail.com,15880597,LV,676915303,89485402,1,NaN,NaN,25JUL2019:04:00:15,27AUG2019:14:36:18,NaN,NO RESPONSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,324,S,"Processed, request sent",N,NaN,27AUG2019:14:36:18,RETAIL_KAC_CC,AVqkPnwjQEKtdINl,riiita@inbox.lv,651869,LV,589934636,89533964,4,NaN,NaN,25JUL2019:04:00:15,27AUG2019:14:36:18,NaN,NO RESPONSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.loc[df["res_STATUS_NAME"] != "NO RESPONSE", :]#.loc[:, ['STATUS','STATUS_NAME','EVENT_TYPE']]
print(df.shape)
df.head()

(20180, 29)


,SEQ_NO,STATUS,STATUS_NAME,EVENT_TYPE,PROCESS_ERROR,SURVEY_EXE_DATE,SURVEY_TYPE,SURVEY_KEY,EMAIL_TO,ID_NO,LANGUAGE,POLICY_NO,POLICY_SEQ_NO,POLICY_YEAR,CLA_CASE_NO,CLA_SUBCASE_NO,CREATED_DATE,TIMESTAMP,res_STATUS,res_STATUS_NAME,res_FEEDBACK_ID,CREATED_AT,COMPLETED_AT,SCORE1,SCORE2,TEXT1,TEXT2,TEXT3,TEXT4
5,331,S,"Processed, request sent",N,NaN,27AUG2019:14:36:18,RETAIL_KAC_CC,XTxyhGdZikeZumyw,dimiva67@mail.ru,1053930,LV,545859467,89533926,6,NaN,NaN,25JUL2019:04:00:15,27AUG2019:14:36:18,N,New,396760850,27AUG2019:20:04:01,27AUG2019:20:04:27,10,NaN,NaN,NaN,NaN,NaN
11,2,S,"Processed, request sent",L,NaN,27AUG2019:14:36:18,RETAIL_CHURN,pXIkIhfdxVFhbShw,aldispr@inbox.lv,765244,LV,601100975,89533758,3,NaN,NaN,25JUL2019:01:04:22,27AUG2019:14:36:18,N,New,398764145,06SEP2019:21:00:06,06SEP2019:21:01:54,7,NaN,NaN,Pagaidām apmierināts ar sadarbību.,Finanses...Budžeta samazināšanās...,NaN
20,42,S,"Processed, request sent",L,NaN,27AUG2019:14:36:20,RETAIL_CHURN,biVgAReMcEfUUvGJ,kiosks1@inbox.lv,56244220,LV,649083541,89534173,2,NaN,NaN,25JUL2019:01:35:40,27AUG2019:14:36:20,N,New,396725225,27AUG2019:16:56:01,27AUG2019:16:57:40,5,NaN,NAV TICIBA APDR KOMP.,NaN,veca masina,NaN
29,164,S,"Processed, request sent",L,NaN,27AUG2019:14:36:23,RETAIL_CHURN,cQdYCXkdRfyIlENk,aivakleikalide@inbox.lv,16105309,LV,654501300,89534441,4,NaN,NaN,25JUL2019:01:42:16,27AUG2019:14:36:23,N,New,396715330,27AUG2019:16:03:24,27AUG2019:16:06:02,6,NaN,"Uzlabot komunikāciju. Ja sola,ka noskaidros un...",NaN,Jo man netika atsūtīts atjaunošanas piedāvājums,NaN
33,220,S,"Processed, request sent",N,NaN,27AUG2019:14:36:24,RETAIL_KAC_CC,xsBDkntBCCcxCTqM,lidija.silina@balta.lv,14028,LV,623198529,84576579,2,NaN,NaN,25JUL2019:04:00:15,27AUG2019:14:36:24,N,New,396701390,27AUG2019:14:40:12,27AUG2019:14:45:44,9,8,NaN,"Uzskatu , ka BALTA tiešām ir godīgākais apdroš...","Uzlabot darbu atlīdzībās. Klienti saka, ka pie...",NaN


In [ ]:
# removing columns with no unique values
df2 = df[[c for c
        in list(df)
        if len(df[c].unique()) > 1]]

In [ ]:
print(df2.shape)
df2.describe()

(20180, 21)


,SEQ_NO,EVENT_TYPE,SURVEY_EXE_DATE,SURVEY_TYPE,SURVEY_KEY,EMAIL_TO,ID_NO,POLICY_NO,POLICY_SEQ_NO,POLICY_YEAR,CREATED_DATE,TIMESTAMP,res_FEEDBACK_ID,CREATED_AT,COMPLETED_AT,SCORE1,SCORE2,TEXT1,TEXT2,TEXT3,TEXT4
count,20180,20180,20180,20180,20180,20180,20180,20180,20180,20180,20180,20180,20180,20180,20180,20180,19272,1116,16783,3600,1491
unique,19773,2,7876,4,19773,17810,18462,19537,19744,18,3710,7877,20180,20041,20056,11,11,988,13631,2771,1138
top,330110,N,05SEP2019:17:21:14,RETAIL_KAC_CC,xmzcBjhirWPztDPe,info@rdbrokers.lv,10092949,685737035,91257890,1,26SEP2019:10:19:17,05SEP2019:17:21:14,439566722,05SEP2019:17:26:38,05SEP2019:17:25:49,10,10,nezinu,Cena,-,Nį
freq,11,19483,13,13235,11,44,11,11,11,13999,197,14,1,4,3,7871,9226,19,292,74,64


In [ ]:
df2.head()

,SEQ_NO,EVENT_TYPE,SURVEY_EXE_DATE,SURVEY_TYPE,SURVEY_KEY,EMAIL_TO,ID_NO,POLICY_NO,POLICY_SEQ_NO,POLICY_YEAR,CREATED_DATE,TIMESTAMP,res_FEEDBACK_ID,CREATED_AT,COMPLETED_AT,SCORE1,SCORE2,TEXT1,TEXT2,TEXT3,TEXT4
5,331,N,27AUG2019:14:36:18,RETAIL_KAC_CC,XTxyhGdZikeZumyw,dimiva67@mail.ru,1053930,545859467,89533926,6,25JUL2019:04:00:15,27AUG2019:14:36:18,396760850,27AUG2019:20:04:01,27AUG2019:20:04:27,10,NaN,NaN,NaN,NaN,NaN
11,2,L,27AUG2019:14:36:18,RETAIL_CHURN,pXIkIhfdxVFhbShw,aldispr@inbox.lv,765244,601100975,89533758,3,25JUL2019:01:04:22,27AUG2019:14:36:18,398764145,06SEP2019:21:00:06,06SEP2019:21:01:54,7,NaN,NaN,Pagaidâm apmierinâts ar sadarbîbu.,Finanses...Budūeta samazinâđanâs...,NaN
20,42,L,27AUG2019:14:36:20,RETAIL_CHURN,biVgAReMcEfUUvGJ,kiosks1@inbox.lv,56244220,649083541,89534173,2,25JUL2019:01:35:40,27AUG2019:14:36:20,396725225,27AUG2019:16:56:01,27AUG2019:16:57:40,5,NaN,NAV TICIBA APDR KOMP.,NaN,veca masina,NaN
29,164,L,27AUG2019:14:36:23,RETAIL_CHURN,cQdYCXkdRfyIlENk,aivakleikalide@inbox.lv,16105309,654501300,89534441,4,25JUL2019:01:42:16,27AUG2019:14:36:23,396715330,27AUG2019:16:03:24,27AUG2019:16:06:02,6,NaN,"Uzlabot komunikâciju. Ja sola,ka noskaidros un...",NaN,Jo man netika atsûtîts atjaunođanas piedâvâjums,NaN
33,220,N,27AUG2019:14:36:24,RETAIL_KAC_CC,xsBDkntBCCcxCTqM,lidija.silina@balta.lv,14028,623198529,84576579,2,25JUL2019:04:00:15,27AUG2019:14:36:24,396701390,27AUG2019:14:40:12,27AUG2019:14:45:44,9,8,NaN,"Uzskatu , ka BALTA tieđâm ir godîgâkais apdrođ...","Uzlabot darbu atlîdzîbâs. Klienti saka, ka pie...",NaN


## Preparing tweet data

In [ ]:
# corp_tweet = pd.read_json("/content/drive/My Drive/BALTA/Sentiment/tweet_corpus_edited.json", orient="str", encoding="ISO-8859-13").loc[:,["text",'sentiment'] ]
# print(corp_tweet.shape)
# corp_tweet.head()


# corp_tweet = pd.read_csv('/content/drive/My Drive/BALTA/Sentiment/tweet_corpus_edited.csv', dtype=str, encoding="ISO-8859-13")
corp_tweet = pd.read_excel('/content/drive/My Drive/BALTA/Sentiment/tweet_corpus_edited.xlsx', encoding="ISO-8859-13")
# corp_tweet = pd.read_csv('/content/drive/My Drive/BALTA/Sentiment/tweets in NPS file.csv', dtype=str, encoding="ISO-8859-13", sep = ",")
corp_tweet.shape
corp_tweet.head()

corp_tweet = corp_tweet.loc[:, ['text','sentiment']]
print(corp_tweet.shape)
corp_tweet.head()

(1176, 2)


,text,sentiment
0,Nu gan laiks gulēt (@ Laines mītne) http://t.c...,NEU
1,"Ja jau neko, nekas neizpalīdz, tad jau jādodas...",NEG
2,es esmu tik šausmīgi dusmīga uz to ka savus mā...,NEG
3,šovakar cirks ar sotiem bija rēcīgs ;D:D diemž...,POZ
4,ļoti bēdīgi palika .,NEG


### Removing special characters

In [ ]:
from textblob import TextBlob

In [ ]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

'x one two'

In [ ]:
corp_tweet_cl = corp_tweet.copy()
corp_tweet_cl.head()

,text,sentiment
0,Nu gan laiks gulēt (@ Laines mītne) http://t.c...,NEU
1,"Ja jau neko, nekas neizpalīdz, tad jau jādodas...",NEG
2,es esmu tik šausmīgi dusmīga uz to ka savus mā...,NEG
3,šovakar cirks ar sotiem bija rēcīgs ;D:D diemž...,POZ
4,ļoti bēdīgi palika .,NEG


In [ ]:
clean_tweet(i[1])

'Man uzbr k'

In [ ]:

for i in enumerate(corp_tweet.iloc[:,0]):
    corp_tweet_cl.iloc[i[0], 0] = clean_tweet(i[1])
corp_tweet_cl.head()

,text,sentiment
0,Nu gan laiks gul t Laines m tne,NEU
1,Ja jau neko nekas neizpal dz tad jau j dodas m...,NEG
2,es esmu tik ausm gi dusm ga uz to ka savus m j...,NEG
3,ovakar cirks ar sotiem bija r c gs D D diem el...,POZ
4,oti b d gi palika,NEG


In [ ]:
import re
string = corp_tweet.loc[0,:] #"aaa\aaa+a.aa"
cleanString = re.sub('\W+','', string )
cleanString

TypeError: ignored

In [ ]:
# corp_tweet.iloc[0,:]
for i in enumerate(corp_tweet.iloc[:,0]):
    print(i)
# for i in enumerate(corp_tweet.iloc[:,0]):
#     print(i)
# corp_tweet.iloc[:,0]

In [ ]:
for i in enumerate(corp_tweet.iloc[0,0].split()):
    new = re.sub('\W+','', i))

Nu
gan
laiks
gulÄt

Laines
mÄtne
httptcoXuuFZ1T5sc


In [ ]:
corp_tweet_cl = corp_tweet.copy()
for i in enumerate(corp_tweet.iloc[:,0]):
    corp_tweet_cl[i[0]] = re.sub('\W+','', i[1])
corp_tweet_cl.head()

In [ ]:
corp_tweet_cl = corp_tweet.copy()
for i in enumerate(corp_tweet.iloc[:,0]):
    corp_tweet_cl.iloc[i[0], 0] = i[1].replace("[comma]", "").lower().replace(".", "").replace("\n", "").replace("'", " ").replace("-", " ").replace("/", " ").replace("?", " ").replace("(", " ").replace(")", " ").strip()     #'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    # corp_tweet_cl[i[0]] = re.sub('\W+','', i[1])
corp_tweet_cl.head()

,text,sentiment
0,nu gan laiks gulät @ laines mä«tne http: t...,NEU
1,"ja jau neko, nekas neizpalä«dz, tad jau jädod...",NEG
2,es esmu tik å”ausmä«gi dusmä«ga uz to ka savus...,NEG
3,å”ovakar cirks ar sotiem bija räcä«gs ;d:d di...,POZ
4,ä¼oti bädä«gi palika,NEG


### Removing stopwords

In [ ]:
stop_words = pd.read_json('/content/drive/My Drive/BALTA/Sentiment/lv stopwords.json', orient="str", encoding="ISO-8859-13")
stop_words.head()

,words
0,aiz
1,ap
2,ar
3,apakÅ”
4,Ärpus


In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()

In [ ]:
def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        #a = enumerate(train_sentences)
        #print(list(a))
        #print('sentence is:', i, sentence)
        new_sent = [lancaster.stem(word) for word in sentence.split() if word not in stop_words]
        print(new_sent)
        train_sentences[i] = ' '.join(new_sent)
    return train_sentences


print("Shape of Tweet Corpus:", corp_tweet.shape)
print('corpus.iloc[2:10] is:\n', corp_tweet.iloc[:10])
print()
corpus_no_stops = filter_stop_words(corp_tweet.iloc[:10, 0], stop_words)
print('\ncorpus_no_stops.iloc[:10]:', corpus_no_stops)

Shape of Tweet Corpus: (1176, 2)
corpus.iloc[2:10] is:
                                                 text sentiment
0  nu gan laik gulät (@ lain mä«tne) http://t.co...       NEU
1  ja jau neko, neka neizpalä«dz, tad jau jädoda...       NEG
2  es esmu tik å”ausmä«gi dusmä«ga uz to ka sav m...       NEG
3  å”ovakar cirk ar sotiem bij räcä«gs ;d:d diem...       POZ
4                             ä¼oti bädä«gi palik .       NEG
5  paldy par forå”o ballä«ti å”odien!;) (@ meitu ...       POZ
6  ir 05:09 un es svinu väl jo projäm.mans sapn...       POZ
7  ä»oti lab pavadä«ju laiku ar @sweetmadaa ulfu ...       POZ
8                                            labrä«t       NEU
9  eju uzpipet, un tad lkm guleet ja jau vis paru...       NEU

['nu', 'gan', 'laik', 'gulä\x93t', '(@', 'lain', 'mä«tne)', 'http://t.co/xuufz1t5sc']
['ja', 'jau', 'neko,', 'nek', 'neizpalä«dz,', 'tad', 'jau', 'jä\x81dodas', 'mä\x81jup!', ':(', '#goodfriends']
['es', 'esmu', 'tik', 'å”ausmä«gi', 'dusmä«ga', 'uz',